# Scikit-Learn Linear Regression
Using SALES_VIEW from DWC. This view has 6,291,450 records

## Install fedml_gcp package

In [ ]:
pip install fedml_gcp

## Import Libraries

In [ ]:
from fedml_gcp import DwcGCP
import numpy as np
import pandas as pd

## Create DwcGCP Instance to access class methods and train model

It is expected that the bucket name passed here already exists in Cloud Storage.

In [ ]:
dwc = DwcGCP(project_name='example-project',
                 bucket_name='<bucket-name>')

### Create tar bundle of script folder so GCP can use it for training

Before running this cell, please ensure that the script package has all the necessary files for a training job.

In [ ]:
dwc.make_tar_bundle('LinearRegression.tar.gz', 'LinearRegression', 'linear/train/LinearRegression.tar.gz')

### Train Model

GCP takes in training inputs that are specific to the training job and the environment needed.

In the training inputs, we are the python module. This is the module that your script package is named, and it references the task.py file inside the script package.

We are also passing args which hold the table name to get data from. Before running the following cell, you should have a config.json file in the script package with the specified values to allow you to access to DWC.

You should also have the follow view `SALES_VIEW` created in your DWC. To gather this data, please refer to https://eforexcel.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/

Please note the 2M records data was downloaded and duplicated 3 times to represent a large dataset in DWC.

In [ ]:
training_inputs = {
    'scaleTier': 'BASIC',
    'packageUris': ['gs://<bucket-name>/linear/train/LinearRegression.tar.gz'],
    'pythonModule': 'trainer.task',
    'args': ['--table_name', 'SALES_VIEW',
             '--table_size', '1',
            '--bucket_name', '<bucket-name>'],
    'region': 'us-east1',
    'jobDir': 'gs://<bucket-name>',
    'runtimeVersion': '2.5',
    'pythonVersion': '3.7',
    'scheduling': {'maxWaitTime': '3600s', 'maxRunningTime': '7200s'}
}

In [ ]:
dwc.train_model('linear_reg', training_inputs)

### Deploy model to Kyma

In [ ]:
dwc.deploy_to_kyma('example@example-project.iam.gserviceaccount.com',  'key.json',  'linear_reg_kyma_deploy', model_location='linear/model/model.pkl')